In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from db_manager import SQLiteWrapper
from functions_kobo import extract_books_and_annotations
from functions_notion import add_book_to_notion, add_annotation_to_notion, get_book_id

In [2]:
sqlite_path = "KoboReader.sqlite"


In [3]:
# Extraer libros y anotaciones
books, book_annotations = extract_books_and_annotations(sqlite_path)


In [5]:
book_annotations

{'La verdad sobre el Caso Savolta': [{'volume_id': 'file:///mnt/onboard/.kobo/dropbox/La_verdad_sobre_el_Caso_Savolta_Eduardo_Mendoza.epub',
   'text': 'una persona honesta, sensible, inteligente y culta.',
   'date': '2023-05-29T18:08:16.000'}],
 'Las madres': [{'volume_id': 'file:///mnt/onboard/.kobo/dropbox/Las_madres_Carmen_Mola.epub',
   'text': 'Buscamos a una mujer de unos veinticinco años, su aspecto ',
   'date': '2023-08-09T21:12:35.587'}],
 'La ciudad de los prodigios': [{'volume_id': 'file:///mnt/onboard/.kobo/dropbox/La_ciudad_de_los_prodigios_Eduardo_Mendoza.epub',
   'text': 'arreboles',
   'date': '2023-12-03T09:42:41.000'},
  {'volume_id': 'file:///mnt/onboard/.kobo/dropbox/La_ciudad_de_los_prodigios_Eduardo_Mendoza.epub',
   'text': 'colores',
   'date': '2023-11-10T17:48:13.000'}],
 'El juego del alma': [{'volume_id': 'file:///mnt/onboard/El_juego_del_alma_Javier_Castillo.epub',
   'text': 'alumnos?\n\n   —Alumna. Suele venir solo con una',
   'date': '2024-08-06T01:

# Paso a paso

In [3]:
db_path = "KoboReader.sqlite"  # Ruta a tu archivo SQLite
db = SQLiteWrapper(db_path)
db.connect()  # Establecer conexión

## Tablas

In [5]:
db_path = "KoboReader.sqlite"  # Ruta a tu archivo SQLite
db = SQLiteWrapper(db_path)
db.connect()  # Establecer conexión



In [9]:
tables_info_df = db.get_tables_info() 

* Shelf: Colecciones de libros. Por ejemplo, Histórica, Thriller, Filosofía...
* ShelfContent: Contenidos de las colecciones de Shelf
* WordList: Lista de palabras guardadas
* DropboxItem: Contenidos de dropbox
* user: Información del usuario. Revisar
* content_settings: configuración de lectura de varios libros
* Achievement: Premios de lectura

Errores de UTF-8:
* SubscriptionProducts
* Activity
* Event

In [13]:
db.get_query_df("select * from user")

,UserID,UserKey,UserDisplayName,UserEmail,___DeviceID,FacebookAuthToken,HasMadePurchase,IsOneStoreAccount,IsChildAccount,RefreshToken,...,Storefront,NewUserPromoCurrency,NewUserPromoValue,KoboAccessToken,KoboAccessTokenExpiry,AnnotationsSyncToken,PrivacyPermissions,AnnotationsMigrated,NotebookSyncTime,NotebookSyncToken
0,83b63413-c6de-45c8-a874-7cadf24716d7,beb19512-af71-45d5-bcb4-fe1ef804aa40,jaguirrepeman@gmail.com,jaguirrepeman@gmail.com,5a9f4f0bf0462fcbefa84fc16badbc1508283270611d49...,None,false,true,false,1R38nbKzczPnuQvyq2yRI25esfR4gsrKvPFO0zVSKoE7hm...,...,ES,None,0.0,eyJhbGciOiJSUzI1NiIsImtpZCI6IjA4NkI4NjkzNUQyQ0...,2025-01-15T08:17:21Z,None,    ,true,2025-01-12T22:02:11.0Z,None


In [14]:
db.get_query_df("select * from Tab")

,tabId,tabType,browseTabType,displayTitle,parentTabId,isDefault,maxSize,totalResults,updateFrequencyMin,imageID,isLeaf,hasFeaturedLists,etag,language
0,SomeFakeWishlistTabID,2,0,None,None,false,0,0,0,None,false,false,"W/""Tm9Db250ZW50""",None


In [15]:
db.get_query_df("select * from KoboPlusAssetGroup")

,Id,AssetGroup,Timestamp,Name,Url,Etag,TimestampTo,Shown
0,77c91545-d80f-46be-86d9-357079015fb2,EPD-KoboPlus-ReadOnly-NeverSubscribed,2021-10-01T00:00:00.0000000Z,EPD-KoboPlus-ReadOnly-NeverSubscribed,https://www.kobo.com/{region}/{language}/plus/...,W/NjM4NTU1MjYxNDkwMDAwMDAwLTU5MDMw,None,true


In [16]:
db.get_query_df("select * from GDriveItem")

,Id,Json,Path
0,10tC86zWmDZ7Inl-Wy26Cj-iSnMkolBVJ,"{\n ""contentHints"": {\n ""thumbnail"":...",/Cómo añadir libros a tu eReader usando Google...


In [17]:
db.get_query_df("select * from Shelf")

,CreationDate,Id,InternalName,LastModified,Name,Type,_IsDeleted,_IsVisible,_IsSynced,_SyncTime,LastAccessed
0,2024-09-03T22:29:45Z,7a58f07d-36d5-45ac-8a05-b8bc873f130c,Filosofía y autoayuda,2024-09-03T22:29:45Z,Filosofía y autoayuda,UserTag,false,true,true,2024-09-05T09:44:53Z,2024-12-29T14:49:42.255
1,2024-09-03T22:29:45Z,79d8a4e4-2ced-40a0-866b-ba46cfd196b0,Histórica,2024-09-03T22:29:45Z,Histórica,UserTag,false,true,true,2024-09-05T09:44:53Z,2025-01-10T18:24:02.220


In [18]:
db.get_query_df("select * from AnalyticsEvents")

,Id,Type,Timestamp,Attributes,Metrics,TestGroups,ClientApplicationVersion,Mandatory
0,c9748c97-fbcd-41f8-bd2e-41d9c8d04e30,TimeToUpdate,2022-01-15T18:49:04Z,{},"{""time"": 109}",{},4.29.18480,false
1,b80ebed0-14c7-4e3a-9f02-0478008ffafb,AppStart,2022-01-15T18:49:05Z,{},{},{},4.29.18480,false
2,1c69fa15-1285-4a76-8e78-30e021417c4b,WelcomeScreen,2022-01-15T18:49:14Z,"{""CurrentLocale"": ""es_ES""}",{},{},4.29.18480,false
3,1cea63da-fb7b-41a7-b7f4-05e7cd0db7a8,WifiSetup,2022-01-15T18:49:17Z,{},{},{},4.29.18480,false


In [19]:
db.get_query_df("select * from SubscriptionProducts")

OperationalError: Could not decode to UTF-8 column 'Tiers' with text ''

In [21]:
db.get_query_df("select * from content_settings limit 5")

,ContentID,ContentType,DateModified,ReadingFontFamily,ReadingFontSize,ReadingAlignment,ReadingLineHeight,ReadingLeftMargin,ReadingRightMargin,ReadingPublisherMode,ActivityFacebookShare,RecentBookSearches,AuthorNotesShown,LastAuthorNotesSyncTime,ZoomFactor,BTBFooterSection,SelectedDictionary,StillReading,SeriesShown
0,file:///mnt/onboard/La_bestia_Carmen_Mola.epub,6,2022-01-15T19:35:15.627,default,28,None,1.35,2,2,None,TRUE,None,false,None,1,None,None,false,false
1,file:///mnt/onboard/.kobo/dropbox/Soy_introver...,6,2022-02-16T20:23:02.739,default,28,None,1.20,2,2,None,TRUE,None,false,None,1,None,None,false,false
2,file:///mnt/onboard/.kobo/dropbox/Los_demonios...,6,2022-08-18T09:27:37.577,default,28,,1.20,2,2,None,TRUE,None,false,None,1,None,None,false,false
3,file:///mnt/onboard/.kobo/dropbox/Cicatriz_Jua...,6,2023-01-13T18:51:18.428,default,30,,1.20,2,2,None,TRUE,None,false,None,1,None,None,false,false
4,file:///mnt/onboard/.kobo/dropbox/Diez_negrito...,6,2023-02-13T00:36:21.842,default,28,None,1.20,2,2,None,TRUE,None,false,None,1,None,None,false,false


In [30]:
db.get_query_df("select * from Achievement")\
        .loc[lambda x: x.PercentComplete= 100].CompleteDescription.values

array(["You're now part of Kobo, and an official Booklover!",
       'You started reading a new book.',
       'You highlighted your first passage.',
       "That's 10 new definitions for you. Now, maybe it's time to go outside?",
       'Make eReading more fun, social and meaningful \\n by building your Reading Life with Kobo. \\n \\n You can track your reading life with interesting \\n statistics and celebrate reading milestones with \\n fun and surprising awards.',
       "You've mastered the art of the long press - the best way to manage your library and dive into the details of a book.",
       'You\'ve finished a book. Is it true that "the good ended happily, and the bad unhappily"?(Thanks to Oscar Wilde for the quote!)',
       "Clearly you couldn't put that book down! You read for two hours straight!",
       "Go big or go home! You've turned 10,000 pages with Kobo."],
      dtype=object)

In [35]:
db.get_query_df("select * from Rules limit 5")

,AchievementId,EventProperty,EventType,GoalValue,Id,Operation,ParentRuleId,ConjunctionType,IsConjunction,Checksum
0,00000000-0000-0000-0000-000000001002,EventCount,27,1,49,1,None,0,0,256e75484f5aad07cb9c9e84b21d0040
1,00000000-0000-0000-0000-000000001008,EventCount,6,1,50,1,None,0,0,b770d31f96b6c145e1902fce4aef671e
2,00000000-0000-0000-0000-00000000100D,EventCount,28,10,51,1,None,0,0,ae35306e2c666c42a76d23acb8a8c48f
3,00000000-0000-0000-0000-000000001000,EventCount,0,1,0,1,None,0,false,b78637ae9596314835a5d2178f92833b
4,00000000-0000-0000-0000-000000001001,EventCount,"35,36",0,1,999,None,1,true,3c707365979a0bba64c589698c77a07e


In [34]:
db.get_query_df("select * from ShelfContent limit 5")

,ShelfName,ContentId,DateModified,_IsDeleted,_IsSynced
0,Filosofía y autoayuda,file:///mnt/onboard/.kobo/dropbox/Mapas_de_sen...,2024-09-02T17:09:39Z,false,false
1,Filosofía y autoayuda,file:///mnt/onboard/.kobo/dropbox/Manipulacion...,2024-09-02T17:09:39Z,false,false
2,Filosofía y autoayuda,file:///mnt/onboard/.kobo/dropbox/12_reglas_pa...,2024-09-02T17:09:39Z,false,false
3,Filosofía y autoayuda,file:///mnt/onboard/.kobo/dropbox/El_Principe_...,2024-09-02T17:09:39Z,false,false
4,Filosofía y autoayuda,file:///mnt/onboard/.kobo/dropbox/30_dias_Camb...,2024-09-02T17:09:39Z,false,false


In [37]:
db.get_query_df("select * from WordList ")

,Text,VolumeId,DictSuffix,DateCreated
0,envarados,file:///mnt/onboard/.kobo/dropbox/Nueva_York_E...,-es,2024-08-20T20:01:28Z
1,argüir,file:///mnt/onboard/.kobo/dropbox/Nueva_York_E...,-es,2024-08-20T21:45:24Z
2,astroso,file:///mnt/onboard/.kobo/dropbox/Ivanhoe_Walt...,-es,2024-08-21T15:11:33Z
3,frívolo,file:///mnt/onboard/.kobo/dropbox/Ivanhoe_Walt...,-es,2024-08-21T15:41:17Z
4,"escabel,",file:///mnt/onboard/.kobo/dropbox/Ivanhoe_Walt...,-es,2024-08-21T15:58:31Z
...,...,...,...,...
78,spiel.,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,-en-es,2025-01-06T08:24:31Z
79,fudge,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,-en-es,2025-01-06T08:26:14Z
80,stifle,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,-en-es,2025-01-08T22:17:02Z
81,stonemason,file:///mnt/onboard/.kobo/dropbox/Surrounded b...,-en-es,2025-01-10T17:56:24Z


In [38]:
db.get_query_df("select * from Activity ")

OperationalError: Could not decode to UTF-8 column 'Data' with text ''

In [42]:
db.get_query_df("select * from DropboxItem").Json[0]

'{\n    "client_modified": "2022-01-30T22:08:16Z",\n    "content_hash": "d8a4640dd72325d32a383da2d0f4d7098bf8ac306851a638dea25c991ebb7e8d",\n    "has_explicit_shared_members": false,\n    "id": "id:3e6PcycqSwUAAAAAAAAAHQ",\n    "media_info": "pending",\n    "name": "Soy_introvertido_Y_que_Steve_Allen.epub",\n    "path_display": "/Soy_introvertido_Y_que_Steve_Allen.epub",\n    "path_lower": "/soy_introvertido_y_que_steve_allen.epub",\n    "property_groups": [\n    ],\n    "rev": "5d6d3e7b0d0b38c86dc51",\n    "server_modified": "2022-01-30T22:08:17Z",\n    "sharing_info": {\n        "read_only": false\n    },\n    "size": "231800"\n}\n'

In [47]:
db.get_query_df("select * from KoboPlusAssets where Language = 'es' ").Value.values

array(['Busca títulos con la etiqueta Kobo Plus en nuestro catálogo.',
       'Añade los eBooks o audiolibros que quieras a tus libros.',
       'Sincroniza tu eReader o aplicación para ver tus libros de Kobo Plus.',
       'Escucha tus audiolibros de Kobo Plus con la aplicación Kobo gratuita para iOS o Android.',
       'Comenzar prueba gratuita',
       'Ideal para descubrir nuevos favoritos y explorar títulos de todos los géneros.',
       '¿Cómo funciona?',
       'https://static.kobo.com/assets/KoboPlusMulti_EPD_CA.svg',
       'Disfruta leyendo o escuchando todo lo que quieras con nuestros planes a partir de 8,99  Є al mes.',
       'Visita kobo.com/plus para más información. Descarga la aplicación gratuita de Kobo en kobo.com/apps.',
       'Comenzar prueba gratuita',
       'Lee y escucha todo lo que quieras con nuestras suscripciones a partir de tan solo 8,99  Є al mes. Elige la que más te convenga.',
       'Más información',
       'https://static.kobo.com/assets/KoboPlusMul

In [49]:
db.get_query_df("select * from Event limit 10 ")

OperationalError: Could not decode to UTF-8 column 'ExtraData' with text ''

In [50]:
db.get_query_df("select * from Bookmark limit 10 ")

OperationalError: Could not decode to UTF-8 column 'ExtraAnnotationData' with text ''

In [51]:
db.get_query_df("select * from volume_shortcovers limit 10 ")

,volumeId,shortcoverId,VolumeIndex
0,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,0
1,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,1
2,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,2
3,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,3
4,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,4
5,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,5
6,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,6
7,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,7
8,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,8
9,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9


In [56]:
db.get_query_df("""select *
                from content limit 10
                """)

,ContentID,ContentType,MimeType,BookID,BookTitle,ImageId,Title,Attribution,Description,DateCreated,...,IsSupported,AnnotationsSyncToken,DateModified,StorePages,StoreWordCount,StoreTimeToReadLowerEstimate,StoreTimeToReadUpperEstimate,Duration,IsAbridged,SyncConflictType
0,22b09ad5-4b26-4130-9424-9cee56aea26f,6,application/x-kobo-epub+zip,None,None,68fd5c91-e759-4154-9b61-eef242623461,Something to Hide,Elizabeth George,<p><strong>Detective Sergeant Barbara Havers a...,2022-01-11T05:00:00.0000000Z,...,true,None,None,0,0,0,0,0,None,0
1,300e8011-45e8-45c8-a010-917c21a983d0,6,application/x-kobo-epub+zip,None,None,1f968fbd-c98a-4880-a02d-aa9cd1f811c6,End of Days,Brad Taylor,<p><strong>Pike Logan must stop a deranged kil...,2022-01-11T05:00:00.0000000Z,...,true,None,None,0,0,0,0,0,None,0
2,6963c490-b680-4e56-aeca-32d8c0dbd50e,6,application/x-kobo-epub+zip,None,None,0c749bd4-38ac-41c5-9d86-0c162c09e4f4,Wahala,Nikki May,"<p><strong>""Utterly winning…brings to mind Sex...",2022-01-11T05:00:00.0000000Z,...,true,None,None,0,0,0,0,0,None,0
3,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Sinopsis,None,None,None,...,true,None,None,0,0,0,0,0,None,0
4,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Portadilla,None,None,None,...,true,None,None,0,0,0,0,0,None,0
5,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,La expresión inglesa...,None,None,None,...,true,None,None,0,0,0,0,0,None,0
6,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Dedicatoria,None,None,None,...,true,None,None,0,0,0,0,0,None,0
7,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Mapa,None,None,None,...,true,None,None,0,0,0,0,0,None,0
8,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,Cita,None,None,None,...,true,None,None,0,0,0,0,0,None,0
9,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,9,application/epub+zip,file:///mnt/onboard/A_fuego_lento_Paula_Hawkin...,A fuego lento,None,"Cubierta de sangre, la chica se adentra...",None,None,None,...,true,None,None,0,0,0,0,0,None,0


In [55]:
db.get_query_df("""select BookID, BookTitle, count(*)
                from content 
                group by 1,2
                order by 3 desc""")

,BookID,BookTitle,count(*)
0,file:///mnt/onboard/.kobo/dropbox/La_grieta_de...,La grieta del silencio,210
1,None,None,180
2,file:///mnt/onboard/.kobo/dropbox/Google Cloud...,Google Cloud Certified Professional Cloud Arch...,127
3,file:///mnt/onboard/.kobo/dropbox/Amor_zero_In...,Amor zero,118
4,file:///mnt/onboard/.kobo/dropbox/En_busca_de_...,En busca de Klingsor,116
...,...,...,...
154,file:///mnt/onboard/.kobo/dropbox/Novela histó...,CAPÍTULO 19,1
155,file:///mnt/onboard/.kobo/dropbox/Novela histó...,CAPÍTULO 20,1
156,file:///mnt/onboard/.kobo/dropbox/Novela histó...,CAPÍTULO 21,1
157,file:///mnt/onboard/.kobo/dropbox/Thing Explai...,Thing Explainer,1


In [11]:
tables_info_df.sort_values("num_filas")

,nombre,num_filas,num_cols,col_names
0,DbVersion,1,1,[version]
14,user,1,29,"[UserID, UserKey, UserDisplayName, UserEmail, ..."
22,Tab,1,14,"[tabId, tabType, browseTabType, displayTitle, ..."
30,KoboPlusAssetGroup,1,8,"[Id, AssetGroup, Timestamp, Name, Url, Etag, T..."
32,GDriveItem,1,3,"[Id, Json, Path]"
13,Shelf,2,11,"[CreationDate, Id, InternalName, LastModified,..."
17,AnalyticsEvents,4,8,"[Id, Type, Timestamp, Attributes, Metrics, Tes..."
28,SubscriptionProducts,4,7,"[CrossRevisionId, Id, Name, IsPreOrder, Tiers,..."
12,content_settings,20,19,"[ContentID, ContentType, DateModified, Reading..."
9,volume_tabs,22,2,"[volumeId, tabId]"


In [15]:
result_df.sort_values("numero_de_filas", ascending = False)

,nombre,numero_de_filas,numero_de_columnas,lista_de_columnas
1,content,6720,101,"[ContentID, ContentType, MimeType, BookID, Boo..."
3,volume_shortcovers,6540,3,"[volumeId, shortcoverId, VolumeIndex]"
5,Bookmark,1201,24,"[BookmarkID, VolumeID, ContentID, StartContain..."
6,Event,1093,7,"[EventType, FirstOccurrence, LastOccurrence, E..."
31,KoboPlusAssets,294,5,"[AssetGroupId, Key, Language, Type, Value]"
29,DropboxItem,154,2,"[Id, Json]"
18,Activity,118,6,"[Id, Enabled, Type, Action, Date, Data]"
26,WordList,83,4,"[Text, VolumeId, DictSuffix, DateCreated]"
11,ShelfContent,66,5,"[ShelfName, ContentId, DateModified, _IsDelete..."
8,Rules,33,10,"[AchievementId, EventProperty, EventType, Goal..."


In [8]:
tables_df.shape

(33, 1)

In [5]:
db.get_query_df("SELECT * from DbVersion")

,version
0,174


In [9]:
db.get_query_df("SELECT * from shortcover_page")

,shortcoverId,PageNumber,FormattedPage


In [ ]:
db.get_query_df("SELECT * from shortcover_page")